In [1]:
from datetime import datetime, timedelta
import numpy as np
from numpy import distutils
import distutils
import pandas as pd
pd.options.plotting.backend = "plotly"
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error
import pickle
import matplotlib.pyplot as plt
import plotly.graph_objects as go


In [2]:
from methods import RM_model
from methods import NN_model
from methods import read_pickle

In [3]:
data_processed = read_pickle('data\data_processed_phase2.pickle')
data_submitted = read_pickle('data\submissions_phase1.pickle')
predictions = data_submitted['submission_Oct9']
ini_predictions = data_submitted['submission_Sep22']

In [6]:
data_processed['weather'].index

DatetimeIndex(['2020-10-31 00:00:00', '2020-10-31 00:15:00',
               '2020-10-31 00:30:00', '2020-10-31 00:45:00',
               '2020-10-31 01:00:00', '2020-10-31 01:15:00',
               '2020-10-31 01:30:00', '2020-10-31 01:45:00',
               '2020-10-31 02:00:00', '2020-10-31 02:15:00',
               ...
               '2020-11-30 21:45:00', '2020-11-30 22:00:00',
               '2020-11-30 22:15:00', '2020-11-30 22:30:00',
               '2020-11-30 22:45:00', '2020-11-30 23:00:00',
               '2020-11-30 23:15:00', '2020-11-30 23:30:00',
               '2020-11-30 23:45:00', '2020-12-01 00:00:00'],
              dtype='datetime64[ns]', name='datetime (UTC)', length=2977, freq=None)

In [9]:
check_key = 'Solar4'
single_user = pd.read_csv('solar4_nov_res.csv',index_col = 0, header = None)
# single_user_cnn = pd.read_csv('solar0_nov.csv',index_col = 0, header = None)
# single_user_101 = pd.read_csv('res_solar0_nov_2.csv',index_col = 0, header = None)
solar4 = single_user.iloc[0]
# solar0_cnn = single_user_cnn.iloc[0]
# solar0_101 = single_user_101.iloc[0]
fig = go.Figure()
time_series = data_processed['weather'].index
fig.add_trace(
        go.Scatter(
            x= time_series, y=solar4, line_width = 0.8, opacity = 0.7, name =check_key + ' resnet prediction'
        )
    )
# fig.add_trace(
#         go.Scatter(
#             x= time_series, y=solar0_cnn, line_width = 0.8, opacity = 0.7, name =check_key + ' cnn prediction'
#         )
#     )
# fig.add_trace(
#         go.Scatter(
#             x= time_series, y=solar0_101, line_width = 0.8, opacity = 0.7, name =check_key + ' resnet101 prediction'
#         )
#     )
if check_key.find('Building') != -1:
    fig.add_scattergl(x=time_series, y=real_value.where(data_processed[check_key]['missing_value'] == 1), line={'color': 'purple'}, name ='invalid values - fixed')
fig.layout.xaxis.title = "Time"
fig.layout.yaxis.title = "ELectricity (kW)"
fig.layout.plot_bgcolor='rgba(0,0,0,0)'
fig.layout.font.family="Times New Roman" ## "Times New Roman Black"
fig.layout.font.size = 20
fig.layout.width = 1000
fig.layout.height = 500
fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True,showgrid=False)
##, tickangle = 45 
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True, showgrid=False)
fig.show()

In [5]:
total_MASE = 0
MSE = []
check_key = 'Building4'
naive_shift = 96*28
for check_key in utilities:
    oct_length = len(predictions[check_key])   
    real_value = data_processed[check_key][check_key][-oct_length:] if check_key.find('Solar') != -1 else data_processed[check_key]['consumption'][-oct_length:]
    naive_value = data_processed[check_key][check_key][-oct_length-naive_shift:-naive_shift] if check_key.find('Solar') != -1 else data_processed[check_key]['consumption'][-oct_length-naive_shift:-naive_shift]
    predictted_value = predictions[check_key]
    real_value = real_value.values
    naive_value = naive_value.values
    predictted_value = predictted_value.values
    if check_key.find('Building') != -1:
        validation = data_processed[check_key]['missing_value'][-oct_length:]
        validation = np.invert(np.array(validation, dtype=bool))
        real_value = real_value[validation]
        naive_value = naive_value[validation]
        predictted_value = predictted_value[validation]
        # print(len(naive_value), len(real_value), len(predictted_value))
    sust = np.mean(np.abs(naive_value - real_value))
    diff = np.mean(np.abs(real_value - predictted_value))
    mase = diff/sust
    MSE.append(diff)
    total_MASE += mase
    print(check_key+' MASE: ' ,diff/sust)
print('average: ', total_MASE/12)

Building0 MASE:  0.7773387813026582
Building1 MASE:  0.8757314815659579
Building3 MASE:  0.642436940677466
Building4 MASE:  0.7107434216076742
Building5 MASE:  0.8100974092971243
Building6 MASE:  0.6682326647888779
Solar0 MASE:  0.7395259988769315
Solar1 MASE:  0.6212096021124899
Solar2 MASE:  0.695794097651066
Solar3 MASE:  0.6718183413045155
Solar4 MASE:  0.6160029378335015
Solar5 MASE:  0.6124795901779192
average:  0.7034509389330151


## For improvements tracking in the phase 1 stage

In [21]:
total_MASE = 0
total_MASE_init = 0
init_MSE = []
MSE = []
naive_shift = 96*28
for check_key in utilities:
    oct_length = len(predictions[check_key])   
    real_value = data_processed[check_key][check_key][-oct_length:] if check_key.find('Solar') != -1 else data_processed[check_key]['consumption'][-oct_length:]
    naive_value = data_processed[check_key][check_key][-oct_length-naive_shift:-naive_shift] if check_key.find('Solar') != -1 else data_processed[check_key]['consumption'][-oct_length-naive_shift:-naive_shift]
    predictted_value = predictions[check_key]
    real_value = real_value.values
    naive_value = naive_value.values
    predictted_value = predictted_value.values
    ini_predictions_value = ini_predictions[check_key].values
    if check_key.find('Building') != -1:
        validation = data_processed[check_key]['missing_value'][-oct_length:]
        validation = np.invert(np.array(validation, dtype=bool))
        real_value = real_value[validation]
        naive_value = naive_value[validation]
        predictted_value = predictted_value[validation]
        ini_predictions_value = ini_predictions_value[validation]
        # print(len(naive_value), len(real_value), len(predictted_value))
    sust = np.mean(np.abs(naive_value - real_value))
    diff = np.mean(np.abs(real_value - predictted_value))
    init_diff = np.mean(np.abs(real_value - ini_predictions_value))
    mase = diff/sust
    init_mase = init_diff/sust
    MSE.append(diff)
    init_MSE.append(init_diff)
    total_MASE += mase
    total_MASE_init += init_mase
    print(check_key+' MASE: ' ,diff/sust)
    print(check_key+' initial MASE: ' ,init_mase)
print('average: ', total_MASE/12)
print('average (init): ', total_MASE_init/12)

Building0 MASE:  0.7773387813026582
Building0 initial MASE:  1.214294776366276
Building1 MASE:  0.8757314815659579
Building1 initial MASE:  0.9015063417562125
Building3 MASE:  0.642436940677466
Building3 initial MASE:  0.6824764665427399
Building4 MASE:  0.7107434216076742
Building4 initial MASE:  0.9239673113458198
Building5 MASE:  0.8100974092971243
Building5 initial MASE:  0.817832553956098
Building6 MASE:  0.6682326647888779
Building6 initial MASE:  0.7751386381168051
Solar0 MASE:  0.7395259988769315
Solar0 initial MASE:  0.7890870084704914
Solar1 MASE:  0.6212096021124899
Solar1 initial MASE:  0.7210342046797881
Solar2 MASE:  0.695794097651066
Solar2 initial MASE:  0.7470473000226338
Solar3 MASE:  0.6718183413045155
Solar3 initial MASE:  0.7371403166805867
Solar4 MASE:  0.6160029378335015
Solar4 initial MASE:  0.7607484808048037
Solar5 MASE:  0.6124795901779192
Solar5 initial MASE:  0.7328141473196604
average:  0.7034509389330151
average (init):  0.8169239621718262


In [23]:
check_key = 'Solar3'
oct_length = len(predictions[check_key])
naive_shift = 96*28
fig = go.Figure()
time_series = data_processed[check_key].index[-oct_length:]
real_value = data_processed[check_key][check_key][-oct_length:] if check_key.find('Solar') != -1 else data_processed[check_key]['consumption'][-oct_length:]
naive_value = data_processed[check_key][check_key][-oct_length-naive_shift:-naive_shift] if check_key.find('Solar') != -1 else data_processed[check_key]['consumption'][-oct_length-naive_shift:-naive_shift]
predictted_value = predictions[check_key]
ini_predictions_value = ini_predictions[check_key]
sust = np.mean(np.abs(naive_value - real_value.values))
diff = np.mean(np.abs(real_value.values - predictted_value))
print('MASE: ' ,diff/sust)
fig.add_trace(
        go.Scatter(
            x= time_series, y=real_value, line_width = 0.8, opacity = 0.7, name =check_key + ' real'
        )
    )
fig.add_trace(
        go.Scatter(
            x= time_series, y=predictted_value, line_width = 0.8, opacity = 0.7, name = check_key + ' predict'
        )
    )
fig.add_trace(
        go.Scatter(
            x= time_series, y=ini_predictions_value, line_width = 0.8, opacity = 0.7, name = check_key + ' initial predict'
        )
    )
if check_key.find('Building') != -1:
    fig.add_scattergl(x=time_series, y=real_value.where(data_processed[check_key]['missing_value'] == 1), line={'color': 'purple'}, name ='invalid values - fixed')
fig.layout.xaxis.title = "Time"
fig.layout.yaxis.title = "ELectricity (kW)"
fig.layout.plot_bgcolor='rgba(0,0,0,0)'
fig.layout.font.family="Times New Roman" ## "Times New Roman Black"
fig.layout.font.size = 20
fig.layout.width = 1000
fig.layout.height = 500
fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True,showgrid=False)
##, tickangle = 45 
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True, showgrid=False)
fig.show()

MASE:  0.6718183413045155


In [38]:
check_key = 'Solar0'
fig = go.Figure()
single_user_res = pd.read_csv('solar0_res.csv',index_col = 0, header = None)
solar1 = single_user_res.iloc[0]
real_value = data_processed[check_key][check_key][-oct_length:] if check_key.find('Solar') != -1 else data_processed[check_key]['consumption'][-oct_length:]
naive_value = data_processed[check_key][check_key][-oct_length-naive_shift:-naive_shift] if check_key.find('Solar') != -1 else data_processed[check_key]['consumption'][-oct_length-naive_shift:-naive_shift]
predictted_value = solar1
# ini_predictions_value = ini_predictions[check_key]
sust = np.mean(np.abs(naive_value - real_value.values))
diff = np.mean(np.abs(real_value.values - predictted_value))
print('Resnet MASE: ' ,diff/sust)
fig.add_trace(
        go.Scatter(
            x= time_series, y=real_value, line_width = 0.8, opacity = 0.7, name =check_key + ' real'
        )
    )
# fig.add_trace(
#         go.Scatter(
#             x= time_series, y=predictted_value, line_width = 0.8, opacity = 0.7, name = check_key + ' predict'
#         )
#     )
fig.add_trace(
        go.Scatter(
            x= time_series, y=predictted_value, line_width = 0.8, opacity = 0.7, name = check_key + ' Resnet'
        )
    )

Resnet MASE:  0.7545464663446442


In [22]:
### Solar 0 : 22222  - 0.71

2880

In [10]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU')]

In [3]:
tf.__version__

'2.3.0'

In [6]:
import sys
sys.version

'3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]'

In [22]:
oct_length

2976

In [28]:
real_oct[-1] 

28.23

In [30]:
extend_value[2975]

28.23

In [32]:
check_key = 'Solar3'
fig = go.Figure()
single_user_res = pd.read_csv('res_solar3_nov_32.csv',index_col = 0, header = None)
time_series_oct = data_processed[check_key].index[-oct_length:].values
time_series = data_processed['weather'].index.values
time_series = np.append(time_series_oct, time_series)
solar1 = single_user_res.iloc[0]
real_oct = data_processed[check_key][check_key][-oct_length:] if check_key.find('Solar') != -1 else data_processed[check_key]['consumption'][-oct_length:]
extend_value = np.append(real_oct, solar1)
time_series
# ini_predictions_value = ini_predictions[check_key]
fig.add_trace(
        go.Scatter(
            x= np.array(range(len(extend_value))), y=extend_value, line_width = 0.8, opacity = 0.7, name =check_key + ' real'
        )
    )
# fig.add_trace(
#         go.Scatter(
#             x= time_series, y=predictted_value, line_width = 0.8, opacity = 0.7, name = check_key + ' predict'
#         )
#     )
# fig.add_trace(
#         go.Scatter(
#             x= time_series, y=predictted_value, line_width = 0.8, opacity = 0.7, name = check_key + ' Resnet'
#         )
#     )
fig.show()

In [ ]:
fig = go.Figure()
single_user_res = pd.read_csv('res_solar4_nov_cnn.csv',index_col = 0, header = None)
time_series = data_processed['weather'].index
solar1 = single_user_res.iloc[0]
predictted_value = solar1
# ini_predictions_value = ini_predictions[check_key]
fig.add_trace(
        go.Scatter(
            x= time_series, y=predictted_value, line_width = 0.8, opacity = 0.7, name =check_key + ' real'
        )
    )
# fig.add_trace(
#         go.Scatter(
#             x= time_series, y=predictted_value, line_width = 0.8, opacity = 0.7, name = check_key + ' predict'
#         )
#     )
# fig.add_trace(
#         go.Scatter(
#             x= time_series, y=predictted_value, line_width = 0.8, opacity = 0.7, name = check_key + ' Resnet'
#         )
#     )
fig.show()